In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import copy

font_path = "C:/Windows/Fonts/MALGUN.TTF"
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)
plt.rc('xtick', labelsize=10)
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
X_train_df_sel = pd.read_csv("../data/dacon_parking_demand_dataset/preprocessed_data/X_train_df.csv")
X_test_df_sel = pd.read_csv("../data/dacon_parking_demand_dataset/preprocessed_data/X_test_df.csv")
y_train_df = pd.read_csv("../data/dacon_parking_demand_dataset/preprocessed_data/y_train_df.csv")
sub = pd.read_csv("../data/dacon_parking_demand_dataset/test.csv")
age = pd.read_csv("../data/dacon_parking_demand_dataset/age_gender_info.csv")

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train_df_sel)

X_train_df_scaled = scaler.transform(X_train_df_sel)
X_test_df_scaled = scaler.transform(X_test_df_sel)
X_test_df_scaled[:5]

array([[-1.71955689, -0.24854825,  0.09980247,  0.1015166 , -0.07385873,
        -0.40235963, -0.6388798 , -1.23153021, -0.08586964, -0.1850583 ,
        -0.49738446, -0.11430211, -0.03235359, -0.05609682,  4.18445665,
        -0.13181763, -0.19297121, -0.10787079, -0.10787079, -0.02641199,
        -0.10279641, -0.01867284, -0.25095121,  0.21128301, -0.26405319,
         1.95495492, -0.38059474, -0.20061544, -0.22734603, -0.19199747,
        -0.31967136, -0.35928128, -0.0918469 , -0.10951231, -0.23238849,
        -0.19872784, -0.18099   , -0.24141474, -0.24543128, -0.49356501,
         0.49356501,  0.03488239,  0.10509177],
       [-1.71837876, -0.24854825, -0.54553172,  0.1015166 ,  0.66893512,
        -0.40235963, -0.6388798 ,  0.81199794, -0.08586964, -0.1850583 ,
        -0.49738446, -0.11430211, -0.03235359, -0.05609682, -0.23897965,
        -0.13181763, -0.19297121, -0.10787079, -0.10787079, -0.02641199,
        -0.10279641, -0.01867284, -0.25095121, -0.55080313, -0.26405319,
   

### 랜덤포레스트회귀 모델 102.80211

In [244]:
# 최고 스코어 모델 -> 피처 42개 적용 및 모델 디폴트 설정으로 학습

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1, random_state=42)
model.fit(X_train_df_scaled, y_train_df)
pred = model.predict(X_test_df_scaled)

result = pd.DataFrame(data=pred, columns=["전용면적 당 차량등록수"], index=test_df.index)
result["단지코드"] = test_df.loc[test_df.index, "단지코드"]

pred_group = result.groupby("단지코드")["전용면적 당 차량등록수"].sum()
final_pred = pd.Series(pred_group)
final_pred = final_pred.reset_index(drop=False)

sub_df = sub.merge(final_pred, how="left")
sub_df = sub_df.drop_duplicates(["단지코드"], keep="first").reset_index()
sub_df = sub_df[["단지코드", "전용면적 당 차량등록수"]]
sub_df = sub_df.rename(columns={"단지코드":"code", "전용면적 당 차량등록수": "num"})
sub_df.to_csv("rf_regressor_with_전용면적_5의배수_전처리_0720.csv", index=False)

sub_df

,code,num
0,C1072,774.758003
1,C1128,1326.299404
2,C1456,564.642018
3,C1840,488.050219
4,C1332,1232.187366
...,...,...
145,C2456,254.745346
146,C1266,393.513184
147,C2152,55.072748
148,C1267,397.565204


In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_train_df_scaled, y_train_df, test_size=0.25,
                                                    random_state=42)

# parameters = {"n_estimators": [100, 200, 300, 500],
#               "max_depth": [None, 3, 5, 7, 10],
#               "min_samples_split": [2, 5, 10, 20]}


# model = RandomForestRegressor(n_jobs=-1, random_state=42)
# grid_model = GridSearchCV(model, param_grid=parameters, cv=3)
# grid_model.fit(X_train, y_train)

In [280]:
pd.DataFrame(grid_model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.465324,1.869882,0.013298,4.701903e-04,None,2,100,"{'max_depth': None, 'min_samples_split': 2, 'n...",0.873588,0.885479,0.858166,0.872411,0.011182,14
1,0.283243,0.017062,0.021941,8.141014e-04,None,2,200,"{'max_depth': None, 'min_samples_split': 2, 'n...",0.874376,0.887882,0.859236,0.873831,0.011701,9
2,0.408242,0.015415,0.031582,4.708084e-04,None,2,300,"{'max_depth': None, 'min_samples_split': 2, 'n...",0.874070,0.888689,0.860382,0.874381,0.011558,7
3,0.690162,0.027047,0.050531,4.705836e-04,None,2,500,"{'max_depth': None, 'min_samples_split': 2, 'n...",0.875467,0.887694,0.861823,0.874995,0.010567,1
4,0.130612,0.002850,0.013500,9.164072e-04,None,5,100,"{'max_depth': None, 'min_samples_split': 5, 'n...",0.870910,0.886362,0.858419,0.871897,0.011429,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.562164,0.028563,0.070146,2.515620e-02,10,10,500,"{'max_depth': 10, 'min_samples_split': 10, 'n_...",0.864658,0.882532,0.863508,0.870233,0.008710,26
76,0.112708,0.002939,0.013146,2.753743e-04,10,20,100,"{'max_depth': 10, 'min_samples_split': 20, 'n_...",0.849821,0.867980,0.865412,0.861071,0.008024,55
77,0.218417,0.027506,0.021608,4.697969e-04,10,20,200,"{'max_depth': 10, 'min_samples_split': 20, 'n_...",0.850048,0.869184,0.866329,0.861854,0.008429,51
78,0.306514,0.028691,0.031915,4.495664e-07,10,20,300,"{'max_depth': 10, 'min_samples_split': 20, 'n_...",0.849483,0.868263,0.864894,0.860880,0.008175,58


In [281]:
grid_model.best_params_

{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 500}

In [283]:
from sklearn.metrics import mean_absolute_error

rf_best = grid_model.best_estimator_

pred = rf_best.predict(X_test)

In [285]:
mean_absolute_error(pred, y_test)

15.950975936777143

In [286]:
rf_model = RandomForestRegressor(n_estimators=500, min_samples_split=2, max_depth=None, random_state=42)
rf_model.fit(X_train_df_scaled, y_train_df)
pred = rf_model.predict(X_test_df_scaled)

result = pd.DataFrame(data=pred, columns=["전용면적 당 차량등록수"], index=test_df.index)
result["단지코드"] = test_df.loc[test_df.index, "단지코드"]

pred_group = result.groupby("단지코드")["전용면적 당 차량등록수"].sum()
final_pred = pd.Series(pred_group)
final_pred = final_pred.reset_index(drop=False)

sub_df = sub.merge(final_pred, how="left")
sub_df = sub_df.drop_duplicates(["단지코드"], keep="first").reset_index()
sub_df = sub_df[["단지코드", "전용면적 당 차량등록수"]]
sub_df = sub_df.rename(columns={"단지코드":"code", "전용면적 당 차량등록수": "num"})
# sub_df.to_csv("rf_regressor_parameter optimization_0719.csv", index=False)

sub_df

,code,num
0,C1072,775.805845
1,C1128,1312.738117
2,C1456,564.213970
3,C1840,494.019631
4,C1332,1207.204622
...,...,...
145,C2456,254.849346
146,C1266,394.142528
147,C2152,51.785465
148,C1267,401.939461


In [33]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(learning_rate=0.1, n_estimators=500, loss="lad", min_samples_split=2, random_state=42)
gb_model.fit(X_train_df_scaled, y_train_df)
pred = gb_model.predict(X_test_df_scaled)

result = pd.DataFrame(data=pred, columns=["전용면적 당 차량등록수"], index=test_df.index)
result["단지코드"] = test_df.loc[test_df.index, "단지코드"]

pred_group = result.groupby("단지코드")["전용면적 당 차량등록수"].sum()
final_pred = pd.Series(pred_group)
final_pred = final_pred.reset_index(drop=False)

sub_df = sub.merge(final_pred, how="left")
sub_df = sub_df.drop_duplicates(["단지코드"], keep="first").reset_index()
sub_df = sub_df[["단지코드", "전용면적 당 차량등록수"]]
sub_df = sub_df.rename(columns={"단지코드":"code", "전용면적 당 차량등록수": "num"})
# sub_df.to_csv("GradientBoostingRegressor_0720.csv", index=False)

sub_df

,code,num
0,C1072,754.981115
1,C1128,1216.313302
2,C1456,559.866057
3,C1840,496.600392
4,C1332,1179.939633
...,...,...
145,C2456,260.811107
146,C1266,400.694082
147,C2152,58.643246
148,C1267,344.265680


In [37]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

gb_model = GradientBoostingRegressor(loss="lad", random_state=42)
parameters = {"learning_rate": [0.01, 0.03, 0.05, 1.0, 2.0, 5.0],
              "n_estimators": [100, 200, 300, 400, 500]}
grid = GridSearchCV(gb_model, param_grid=parameters, cv=3)
grid.fit(X_train, y_train)

C:\Users\HYUNJUN\anaconda3\envs\likelion\lib\site-packages\sklearn\metrics\_regression.py:691: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,
C:\Users\HYUNJUN\anaconda3\envs\likelion\lib\site-packages\sklearn\metrics\_regression.py:691: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,
C:\Users\HYUNJUN\anaconda3\envs\likelion\lib\site-packages\sklearn\metrics\_regression.py:691: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,
C:\Users\HYUNJUN\anaconda3\envs\likelion\lib\site-packages\sklearn\metrics\_regression.py:691: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,
C:\Users\HYUNJUN\anaconda3\envs\likelion\lib\site-packages\sklearn\metrics\_regression.py:691: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).su

GridSearchCV(cv=3,
             estimator=GradientBoostingRegressor(loss='lad', random_state=42),
             param_grid={'learning_rate': [0.01, 0.03, 0.05, 1.0, 2.0, 5.0],
                         'n_estimators': [100, 200, 300, 400, 500]})

In [38]:
print(grid.best_params_)
gb_best = grid.best_estimator_
pred = gb_best.predict(X_test_df_scaled)

result = pd.DataFrame(data=pred, columns=["전용면적 당 차량등록수"], index=test_df.index)
result["단지코드"] = test_df.loc[test_df.index, "단지코드"]

pred_group = result.groupby("단지코드")["전용면적 당 차량등록수"].sum()
final_pred = pd.Series(pred_group)
final_pred = final_pred.reset_index(drop=False)

sub_df = sub.merge(final_pred, how="left")
sub_df = sub_df.drop_duplicates(["단지코드"], keep="first").reset_index()
sub_df = sub_df[["단지코드", "전용면적 당 차량등록수"]]
sub_df = sub_df.rename(columns={"단지코드":"code", "전용면적 당 차량등록수": "num"})
# sub_df.to_csv("GradientBoostingRegressor with gridsearch_0720.csv", index=False)

sub_df

{'learning_rate': 0.05, 'n_estimators': 500}


,code,num
0,C1072,749.252101
1,C1128,1298.537068
2,C1456,508.921070
3,C1840,506.033658
4,C1332,1161.370868
...,...,...
145,C2456,240.474021
146,C1266,424.966866
147,C2152,65.289823
148,C1267,390.044439


In [337]:
model = RandomForestRegressor(n_jobs=-1, random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)

mean_absolute_error(pred, y_test)

15.994010876279114

In [338]:
from sklearn.linear_model import Ridge, Lasso

parameters = {"alpha": [0.01, 0.1, 0.2, 0.5, 1, 2, 5, 10]}
ridge_model = Ridge()

grid_model = GridSearchCV(ridge_model, param_grid=parameters, cv=3)
grid_model.fit(X_train, y_train)

grid_model.best_params_

{'alpha': 5}

In [343]:
ridge_best = grid_model.best_estimator_
pred = ridge_best.predict(X_test)

mean_absolute_error(pred, y_test)

20.25568295624673